In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as st
from scipy.stats import linregress
from IPython.display import display, Javascript
import re

%matplotlib widget
pd.options.display.max_columns = None

import warnings
warnings.filterwarnings('ignore')

# Study data files
ed_data = "data/acs_2020_s1501.csv"

# Read the mouse data and the study results
ed_data_raw = pd.read_csv(ed_data)

In [2]:
# Display the data tables for preview
ed_data_raw.head(20)

,Label (Grouping),RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT,RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!White alone,RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!White alone!!High school graduate or higher,RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!White alone!!Bachelor's degree or higher,"RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!White alone, not Hispanic or Latino","RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!White alone, not Hispanic or Latino!!High school graduate or higher","RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!White alone, not Hispanic or Latino!!Bachelor's degree or higher",RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!Black alone,RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!Black alone!!High school graduate or higher,RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!Black alone!!Bachelor's degree or higher,RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!American Indian or Alaska Native alone,RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!American Indian or Alaska Native alone!!High school graduate or higher,RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!American Indian or Alaska Native alone!!Bachelor's degree or higher,RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!Asian alone,RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!Asian alone!!High school graduate or higher,RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!Asian alone!!Bachelor's degree or higher,RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!Native Hawaiian and Other Pacific Islander alone,RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!Native Hawaiian and Other Pacific Islander alone!!High school graduate or higher,RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!Native Hawaiian and Other Pacific Islander alone!!Bachelor's degree or higher,RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!Some other race alone,RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!Some other race alone!!High school graduate or higher,RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!Some other race alone!!Bachelor's degree or higher,RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!Two or more races,RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!Two or more races!!High school graduate or higher,RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!Two or more races!!Bachelor's degree or higher,RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!Hispanic or Latino Origin,RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!Hispanic or Latino Origin!!High school graduate or higher,RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!Hispanic or Latino Origin!!Bachelor's degree or higher
0,"Census Tract 1.01, District of Columbia, Distr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Estimate,NaN,"1,016",993,905,867,867,779,0,0,0,0,0,0,54,54,54,0,0,0,14,14,14,2,2,2,151,128,128
3,Margin of Error,NaN,±203,±202,±192,±202,±202,±192,±13,±13,±13,±13,±13,±13,±68,±68,±68,±13,±13,±13,±22,±22,±22,±4,±4,±4,±131,±127,±127
4,Percent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Estimate,NaN,(X),97.70%,89.10%,(X),100.00%,89.90%,(X),-,-,(X),-,-,(X),100.00%,100.00%,(X),-,-,(X),100.00%,100.00%,(X),100.00%,100.00%,(X),84.80%,84.80%
6,Margin of Error,NaN,(X),±3.5,±6.9,(X),±4.5,±7.5,(X),**,**,(X),**,**,(X),±44.8,±44.8,(X),**,**,(X),±87.9,±87.9,(X),±100.0,±100.0,(X),±26.7,±26.7
7,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [3]:
# Rename column #1
ed_data_raw = ed_data_raw.rename(columns={"Label (Grouping)": "label"})

# Reduce length of labels for columns (educational data)
ed_data_raw.columns = ed_data_raw.columns.map(lambda x: x.replace("RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT", ""))
ed_data_raw.columns = ed_data_raw.columns.map(lambda x: x.replace("!!High school graduate or higher", "_hs"))
ed_data_raw.columns = ed_data_raw.columns.map(lambda x: x.replace("!!Bachelor's degree or higher", "_col"))

# Reduce length of labels for columns (race data)
ed_data_raw.columns = ed_data_raw.columns.map(lambda x: x.replace("!!White alone", "white"))
ed_data_raw.columns = ed_data_raw.columns.map(lambda x: x.replace("white, not Hispanic or Latino", "white-nonhisp"))
ed_data_raw.columns = ed_data_raw.columns.map(lambda x: x.replace("!!American Indian or Alaska Native alone", "aian"))
ed_data_raw.columns = ed_data_raw.columns.map(lambda x: x.replace("!!Black alone", "black"))
ed_data_raw.columns = ed_data_raw.columns.map(lambda x: x.replace("!!Native Hawaiian and Other Pacific Islander alone", "nhpi"))
ed_data_raw.columns = ed_data_raw.columns.map(lambda x: x.replace("!!Asian alone", "asian"))
ed_data_raw.columns = ed_data_raw.columns.map(lambda x: x.replace("!!Some other race alone", "other"))
ed_data_raw.columns = ed_data_raw.columns.map(lambda x: x.replace("!!Two or more races", "multi"))
ed_data_raw.columns = ed_data_raw.columns.map(lambda x: x.replace("!!Hispanic or Latino Origin", "hisp"))

# Display the updated data table
ed_data_raw.head(10)

,label,,white,white_hs,white_col,white-nonhisp,white-nonhisp_hs,white-nonhisp_col,black,black_hs,black_col,aian,aian_hs,aian_col,asian,asian_hs,asian_col,nhpi,nhpi_hs,nhpi_col,other,other_hs,other_col,multi,multi_hs,multi_col,hisp,hisp_hs,hisp_col
0,"Census Tract 1.01, District of Columbia, Distr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Estimate,NaN,"1,016",993,905,867,867,779,0,0,0,0,0,0,54,54,54,0,0,0,14,14,14,2,2,2,151,128,128
3,Margin of Error,NaN,±203,±202,±192,±202,±202,±192,±13,±13,±13,±13,±13,±13,±68,±68,±68,±13,±13,±13,±22,±22,±22,±4,±4,±4,±131,±127,±127
4,Percent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Estimate,NaN,(X),97.70%,89.10%,(X),100.00%,89.90%,(X),-,-,(X),-,-,(X),100.00%,100.00%,(X),-,-,(X),100.00%,100.00%,(X),100.00%,100.00%,(X),84.80%,84.80%
6,Margin of Error,NaN,(X),±3.5,±6.9,(X),±4.5,±7.5,(X),**,**,(X),**,**,(X),±44.8,±44.8,(X),**,**,(X),±87.9,±87.9,(X),±100.0,±100.0,(X),±26.7,±26.7
7,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Estimate,NaN,463,440,406,370,370,336,0,0,0,0,0,0,2,2,2,0,0,0,14,14,14,2,2,2,95,72,72
9,Margin of Error,NaN,±143,±134,±124,±117,±117,±109,±13,±13,±13,±13,±13,±13,±36,±36,±36,±13,±13,±13,±22,±22,±22,±4,±4,±4,±79,±71,±71


In [4]:
# Iterate through all the rows in the table to clean the "label" column and remove any additonal spaces retained from CSV
length = ed_data_raw.shape[0]
for i in range(length):
    ed_data_raw["label"].iloc[i] = ed_data_raw["label"].iloc[i].strip()

# Display the updated data table
ed_data_raw.head(10)

,label,,white,white_hs,white_col,white-nonhisp,white-nonhisp_hs,white-nonhisp_col,black,black_hs,black_col,aian,aian_hs,aian_col,asian,asian_hs,asian_col,nhpi,nhpi_hs,nhpi_col,other,other_hs,other_col,multi,multi_hs,multi_col,hisp,hisp_hs,hisp_col
0,"Census Tract 1.01, District of Columbia, Distr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Estimate,NaN,"1,016",993,905,867,867,779,0,0,0,0,0,0,54,54,54,0,0,0,14,14,14,2,2,2,151,128,128
3,Margin of Error,NaN,±203,±202,±192,±202,±202,±192,±13,±13,±13,±13,±13,±13,±68,±68,±68,±13,±13,±13,±22,±22,±22,±4,±4,±4,±131,±127,±127
4,Percent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Estimate,NaN,(X),97.70%,89.10%,(X),100.00%,89.90%,(X),-,-,(X),-,-,(X),100.00%,100.00%,(X),-,-,(X),100.00%,100.00%,(X),100.00%,100.00%,(X),84.80%,84.80%
6,Margin of Error,NaN,(X),±3.5,±6.9,(X),±4.5,±7.5,(X),**,**,(X),**,**,(X),±44.8,±44.8,(X),**,**,(X),±87.9,±87.9,(X),±100.0,±100.0,(X),±26.7,±26.7
7,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Estimate,NaN,463,440,406,370,370,336,0,0,0,0,0,0,2,2,2,0,0,0,14,14,14,2,2,2,95,72,72
9,Margin of Error,NaN,±143,±134,±124,±117,±117,±109,±13,±13,±13,±13,±13,±13,±36,±36,±36,±13,±13,±13,±22,±22,±22,±4,±4,±4,±79,±71,±71


In [5]:
# Iterate through all the rows in the table to rename the "Estimate" rows with unique values under each census tract
for i in range(length):
    if ed_data_raw["label"].iloc[i] == "Estimate":
        ed_data_raw["label"].iloc[i] = "Pop_" + ed_data_raw["label"].iloc[i-1]

# Display the updated data table
ed_data_raw.head(20)

,label,,white,white_hs,white_col,white-nonhisp,white-nonhisp_hs,white-nonhisp_col,black,black_hs,black_col,aian,aian_hs,aian_col,asian,asian_hs,asian_col,nhpi,nhpi_hs,nhpi_col,other,other_hs,other_col,multi,multi_hs,multi_col,hisp,hisp_hs,hisp_col
0,"Census Tract 1.01, District of Columbia, Distr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Pop_Total,NaN,"1,016",993,905,867,867,779,0,0,0,0,0,0,54,54,54,0,0,0,14,14,14,2,2,2,151,128,128
3,Margin of Error,NaN,±203,±202,±192,±202,±202,±192,±13,±13,±13,±13,±13,±13,±68,±68,±68,±13,±13,±13,±22,±22,±22,±4,±4,±4,±131,±127,±127
4,Percent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Pop_Percent,NaN,(X),97.70%,89.10%,(X),100.00%,89.90%,(X),-,-,(X),-,-,(X),100.00%,100.00%,(X),-,-,(X),100.00%,100.00%,(X),100.00%,100.00%,(X),84.80%,84.80%
6,Margin of Error,NaN,(X),±3.5,±6.9,(X),±4.5,±7.5,(X),**,**,(X),**,**,(X),±44.8,±44.8,(X),**,**,(X),±87.9,±87.9,(X),±100.0,±100.0,(X),±26.7,±26.7
7,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Pop_Male,NaN,463,440,406,370,370,336,0,0,0,0,0,0,2,2,2,0,0,0,14,14,14,2,2,2,95,72,72
9,Margin of Error,NaN,±143,±134,±124,±117,±117,±109,±13,±13,±13,±13,±13,±13,±36,±36,±36,±13,±13,±13,±22,±22,±22,±4,±4,±4,±79,±71,±71


In [6]:
# Delete any rows that contain the query text
ed_data_raw = ed_data_raw[~ed_data_raw["label"].str.contains('Margin of Error')]
ed_data_raw = ed_data_raw[~ed_data_raw["label"].str.contains('Percent')]

# Delete any rows that start with the query text
ed_data_raw = ed_data_raw[~ed_data_raw["label"].str.startswith('Total')]
ed_data_raw = ed_data_raw[~ed_data_raw["label"].str.startswith('Male')]
ed_data_raw = ed_data_raw[~ed_data_raw["label"].str.startswith('Female')]

# Delete unneeded columns corresponding to White Non-Hispanic
ed_data_raw.drop(['white-nonhisp', 'white-nonhisp_hs', 'white-nonhisp_col'], axis=1, inplace=True)

# Display the updated data table
ed_data_raw.head(20)

,label,,white,white_hs,white_col,black,black_hs,black_col,aian,aian_hs,aian_col,asian,asian_hs,asian_col,nhpi,nhpi_hs,nhpi_col,other,other_hs,other_col,multi,multi_hs,multi_col,hisp,hisp_hs,hisp_col
0,"Census Tract 1.01, District of Columbia, Distr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Pop_Total,NaN,"1,016",993,905,0,0,0,0,0,0,54,54,54,0,0,0,14,14,14,2,2,2,151,128,128
8,Pop_Male,NaN,463,440,406,0,0,0,0,0,0,2,2,2,0,0,0,14,14,14,2,2,2,95,72,72
14,Pop_Female,NaN,553,553,499,0,0,0,0,0,0,52,52,52,0,0,0,0,0,0,0,0,0,56,56,56
19,"Census Tract 1.02, District of Columbia, Distr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,Pop_Total,NaN,"2,519","2,501","2,317",34,34,16,0,0,0,118,97,87,0,0,0,147,134,134,94,94,94,332,301,265
27,Pop_Male,NaN,"1,345","1,345","1,200",0,0,0,0,0,0,19,19,9,0,0,0,57,57,57,84,84,84,209,209,173
33,Pop_Female,NaN,"1,174","1,156","1,117",34,34,16,0,0,0,99,78,78,0,0,0,90,77,77,10,10,10,123,92,92
38,"Census Tract 2.01, District of Columbia, Distr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,Pop_Total,NaN,48,48,44,9,9,0,0,0,0,30,30,30,0,0,0,0,0,0,0,0,0,8,8,8


In [7]:
# Rename column #2
ed_data_raw = ed_data_raw.rename(columns={ed_data_raw.columns[1]: "tract"})

# Display the updated data table
ed_data_raw.head(20)

,label,tract,white,white_hs,white_col,black,black_hs,black_col,aian,aian_hs,aian_col,asian,asian_hs,asian_col,nhpi,nhpi_hs,nhpi_col,other,other_hs,other_col,multi,multi_hs,multi_col,hisp,hisp_hs,hisp_col
0,"Census Tract 1.01, District of Columbia, Distr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Pop_Total,NaN,"1,016",993,905,0,0,0,0,0,0,54,54,54,0,0,0,14,14,14,2,2,2,151,128,128
8,Pop_Male,NaN,463,440,406,0,0,0,0,0,0,2,2,2,0,0,0,14,14,14,2,2,2,95,72,72
14,Pop_Female,NaN,553,553,499,0,0,0,0,0,0,52,52,52,0,0,0,0,0,0,0,0,0,56,56,56
19,"Census Tract 1.02, District of Columbia, Distr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,Pop_Total,NaN,"2,519","2,501","2,317",34,34,16,0,0,0,118,97,87,0,0,0,147,134,134,94,94,94,332,301,265
27,Pop_Male,NaN,"1,345","1,345","1,200",0,0,0,0,0,0,19,19,9,0,0,0,57,57,57,84,84,84,209,209,173
33,Pop_Female,NaN,"1,174","1,156","1,117",34,34,16,0,0,0,99,78,78,0,0,0,90,77,77,10,10,10,123,92,92
38,"Census Tract 2.01, District of Columbia, Distr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,Pop_Total,NaN,48,48,44,9,9,0,0,0,0,30,30,30,0,0,0,0,0,0,0,0,0,8,8,8


In [8]:
# Iterate through all the rows in the table to extract the census tract number and add it to the "tract" column
for index, row in ed_data_raw.iterrows():
    if row[0].startswith('Census'):
        text = row[0]
        tract = float(''.join(re.findall('[0-9]*\.?[0-9]*', text)))
        tract = tract * 100
        ed_data_raw.loc[index, "tract"] = tract

# Display the updated data table
ed_data_raw.head(20)

,label,tract,white,white_hs,white_col,black,black_hs,black_col,aian,aian_hs,aian_col,asian,asian_hs,asian_col,nhpi,nhpi_hs,nhpi_col,other,other_hs,other_col,multi,multi_hs,multi_col,hisp,hisp_hs,hisp_col
0,"Census Tract 1.01, District of Columbia, Distr...",101.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Pop_Total,NaN,"1,016",993,905,0,0,0,0,0,0,54,54,54,0,0,0,14,14,14,2,2,2,151,128,128
8,Pop_Male,NaN,463,440,406,0,0,0,0,0,0,2,2,2,0,0,0,14,14,14,2,2,2,95,72,72
14,Pop_Female,NaN,553,553,499,0,0,0,0,0,0,52,52,52,0,0,0,0,0,0,0,0,0,56,56,56
19,"Census Tract 1.02, District of Columbia, Distr...",102.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,Pop_Total,NaN,"2,519","2,501","2,317",34,34,16,0,0,0,118,97,87,0,0,0,147,134,134,94,94,94,332,301,265
27,Pop_Male,NaN,"1,345","1,345","1,200",0,0,0,0,0,0,19,19,9,0,0,0,57,57,57,84,84,84,209,209,173
33,Pop_Female,NaN,"1,174","1,156","1,117",34,34,16,0,0,0,99,78,78,0,0,0,90,77,77,10,10,10,123,92,92
38,"Census Tract 2.01, District of Columbia, Distr...",201.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,Pop_Total,NaN,48,48,44,9,9,0,0,0,0,30,30,30,0,0,0,0,0,0,0,0,0,8,8,8


In [9]:
ed_data_raw.loc[:,"tract"] = ed_data_raw.loc[:,"tract"].ffill()

# Display the updated data table
ed_data_raw.head(20)

,label,tract,white,white_hs,white_col,black,black_hs,black_col,aian,aian_hs,aian_col,asian,asian_hs,asian_col,nhpi,nhpi_hs,nhpi_col,other,other_hs,other_col,multi,multi_hs,multi_col,hisp,hisp_hs,hisp_col
0,"Census Tract 1.01, District of Columbia, Distr...",101.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Pop_Total,101.0,"1,016",993,905,0,0,0,0,0,0,54,54,54,0,0,0,14,14,14,2,2,2,151,128,128
8,Pop_Male,101.0,463,440,406,0,0,0,0,0,0,2,2,2,0,0,0,14,14,14,2,2,2,95,72,72
14,Pop_Female,101.0,553,553,499,0,0,0,0,0,0,52,52,52,0,0,0,0,0,0,0,0,0,56,56,56
19,"Census Tract 1.02, District of Columbia, Distr...",102.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,Pop_Total,102.0,"2,519","2,501","2,317",34,34,16,0,0,0,118,97,87,0,0,0,147,134,134,94,94,94,332,301,265
27,Pop_Male,102.0,"1,345","1,345","1,200",0,0,0,0,0,0,19,19,9,0,0,0,57,57,57,84,84,84,209,209,173
33,Pop_Female,102.0,"1,174","1,156","1,117",34,34,16,0,0,0,99,78,78,0,0,0,90,77,77,10,10,10,123,92,92
38,"Census Tract 2.01, District of Columbia, Distr...",201.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,Pop_Total,201.0,48,48,44,9,9,0,0,0,0,30,30,30,0,0,0,0,0,0,0,0,0,8,8,8


In [10]:
ed_data_raw["tract"] = ed_data_raw["tract"].astype(int)

# Display the updated data table
ed_data_raw.head(20)

,label,tract,white,white_hs,white_col,black,black_hs,black_col,aian,aian_hs,aian_col,asian,asian_hs,asian_col,nhpi,nhpi_hs,nhpi_col,other,other_hs,other_col,multi,multi_hs,multi_col,hisp,hisp_hs,hisp_col
0,"Census Tract 1.01, District of Columbia, Distr...",101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Pop_Total,101,"1,016",993,905,0,0,0,0,0,0,54,54,54,0,0,0,14,14,14,2,2,2,151,128,128
8,Pop_Male,101,463,440,406,0,0,0,0,0,0,2,2,2,0,0,0,14,14,14,2,2,2,95,72,72
14,Pop_Female,101,553,553,499,0,0,0,0,0,0,52,52,52,0,0,0,0,0,0,0,0,0,56,56,56
19,"Census Tract 1.02, District of Columbia, Distr...",102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,Pop_Total,102,"2,519","2,501","2,317",34,34,16,0,0,0,118,97,87,0,0,0,147,134,134,94,94,94,332,301,265
27,Pop_Male,102,"1,345","1,345","1,200",0,0,0,0,0,0,19,19,9,0,0,0,57,57,57,84,84,84,209,209,173
33,Pop_Female,102,"1,174","1,156","1,117",34,34,16,0,0,0,99,78,78,0,0,0,90,77,77,10,10,10,123,92,92
38,"Census Tract 2.01, District of Columbia, Distr...",200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,Pop_Total,200,48,48,44,9,9,0,0,0,0,30,30,30,0,0,0,0,0,0,0,0,0,8,8,8


In [11]:
# Delete any rows that start with the query text
ed_data_raw = ed_data_raw[~ed_data_raw["label"].str.startswith('Census')]

# Display the updated data table
ed_data_raw.head(20)

,label,tract,white,white_hs,white_col,black,black_hs,black_col,aian,aian_hs,aian_col,asian,asian_hs,asian_col,nhpi,nhpi_hs,nhpi_col,other,other_hs,other_col,multi,multi_hs,multi_col,hisp,hisp_hs,hisp_col
2,Pop_Total,101,"1,016",993,905,0,0,0,0,0,0,54,54,54,0,0,0,14,14,14,2,2,2,151,128,128
8,Pop_Male,101,463,440,406,0,0,0,0,0,0,2,2,2,0,0,0,14,14,14,2,2,2,95,72,72
14,Pop_Female,101,553,553,499,0,0,0,0,0,0,52,52,52,0,0,0,0,0,0,0,0,0,56,56,56
21,Pop_Total,102,"2,519","2,501","2,317",34,34,16,0,0,0,118,97,87,0,0,0,147,134,134,94,94,94,332,301,265
27,Pop_Male,102,"1,345","1,345","1,200",0,0,0,0,0,0,19,19,9,0,0,0,57,57,57,84,84,84,209,209,173
33,Pop_Female,102,"1,174","1,156","1,117",34,34,16,0,0,0,99,78,78,0,0,0,90,77,77,10,10,10,123,92,92
40,Pop_Total,200,48,48,44,9,9,0,0,0,0,30,30,30,0,0,0,0,0,0,0,0,0,8,8,8
46,Pop_Male,200,48,48,44,2,2,0,0,0,0,6,6,6,0,0,0,0,0,0,0,0,0,8,8,8
52,Pop_Female,200,0,0,0,7,7,0,0,0,0,24,24,24,0,0,0,0,0,0,0,0,0,0,0,0
59,Pop_Total,202,"2,684","2,669","2,356",47,47,18,0,0,0,112,112,112,18,18,18,81,81,81,12,12,12,175,175,175


In [12]:
# Review dtypes in each column; note, right now most are "objects" that do not allow for calculation
ed_data_raw.dtypes

label        object
tract         int64
white        object
white_hs     object
white_col    object
black        object
black_hs     object
black_col    object
aian         object
aian_hs      object
aian_col     object
asian        object
asian_hs     object
asian_col    object
nhpi         object
nhpi_hs      object
nhpi_col     object
other        object
other_hs     object
other_col    object
multi        object
multi_hs     object
multi_col    object
hisp         object
hisp_hs      object
hisp_col     object
dtype: object

In [13]:
# Reset index
ed_data_raw = ed_data_raw.set_index(["label"])

# Apply regex in place to remove commas from all values
ed_data_raw = ed_data_raw.replace(',','', regex=True)

# Convert all dtypes from object to int64 to allow for calucation
ed_data_raw = ed_data_raw.apply(pd.to_numeric, errors='coerce')

# Display the updated data table
ed_data_raw.head(20)

,tract,white,white_hs,white_col,black,black_hs,black_col,aian,aian_hs,aian_col,asian,asian_hs,asian_col,nhpi,nhpi_hs,nhpi_col,other,other_hs,other_col,multi,multi_hs,multi_col,hisp,hisp_hs,hisp_col
label,,,,,,,,,,,,,,,,,,,,,,,,,
Pop_Total,101,1016,993,905,0,0,0,0,0,0,54,54,54,0,0,0,14,14,14,2,2,2,151,128,128
Pop_Male,101,463,440,406,0,0,0,0,0,0,2,2,2,0,0,0,14,14,14,2,2,2,95,72,72
Pop_Female,101,553,553,499,0,0,0,0,0,0,52,52,52,0,0,0,0,0,0,0,0,0,56,56,56
Pop_Total,102,2519,2501,2317,34,34,16,0,0,0,118,97,87,0,0,0,147,134,134,94,94,94,332,301,265
Pop_Male,102,1345,1345,1200,0,0,0,0,0,0,19,19,9,0,0,0,57,57,57,84,84,84,209,209,173
Pop_Female,102,1174,1156,1117,34,34,16,0,0,0,99,78,78,0,0,0,90,77,77,10,10,10,123,92,92
Pop_Total,200,48,48,44,9,9,0,0,0,0,30,30,30,0,0,0,0,0,0,0,0,0,8,8,8
Pop_Male,200,48,48,44,2,2,0,0,0,0,6,6,6,0,0,0,0,0,0,0,0,0,8,8,8
Pop_Female,200,0,0,0,7,7,0,0,0,0,24,24,24,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
# Review dtypes in each column; note, now all are "int64" which allows for calculation
ed_data_raw.dtypes

tract        int64
white        int64
white_hs     int64
white_col    int64
black        int64
black_hs     int64
black_col    int64
aian         int64
aian_hs      int64
aian_col     int64
asian        int64
asian_hs     int64
asian_col    int64
nhpi         int64
nhpi_hs      int64
nhpi_col     int64
other        int64
other_hs     int64
other_col    int64
multi        int64
multi_hs     int64
multi_col    int64
hisp         int64
hisp_hs      int64
hisp_col     int64
dtype: object

In [15]:
# Calculate number of individauls from each group who have less than a high school education and insert as columns into table
ed_data_raw.insert(2, 'white_no', ed_data_raw["white"] - ed_data_raw["white_hs"])
ed_data_raw.insert(6, 'black_no', ed_data_raw["black"] - ed_data_raw["black_hs"])
ed_data_raw.insert(10, 'aian_no', ed_data_raw["aian"] - ed_data_raw["aian_hs"])
ed_data_raw.insert(14, 'asian_no', ed_data_raw["asian"] - ed_data_raw["asian_hs"])
ed_data_raw.insert(18, 'nhpi_no', ed_data_raw["nhpi"] - ed_data_raw["nhpi_hs"])
ed_data_raw.insert(22, 'other_no', ed_data_raw["other"] - ed_data_raw["other_hs"])
ed_data_raw.insert(26, 'multi_no', ed_data_raw["multi"] - ed_data_raw["multi_hs"])
ed_data_raw.insert(30, 'hisp_no', ed_data_raw["hisp"] - ed_data_raw["hisp_hs"])

# Display the updated data table
ed_data_raw.head(20)

,tract,white,white_no,white_hs,white_col,black,black_no,black_hs,black_col,aian,aian_no,aian_hs,aian_col,asian,asian_no,asian_hs,asian_col,nhpi,nhpi_no,nhpi_hs,nhpi_col,other,other_no,other_hs,other_col,multi,multi_no,multi_hs,multi_col,hisp,hisp_no,hisp_hs,hisp_col
label,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Pop_Total,101,1016,23,993,905,0,0,0,0,0,0,0,0,54,0,54,54,0,0,0,0,14,0,14,14,2,0,2,2,151,23,128,128
Pop_Male,101,463,23,440,406,0,0,0,0,0,0,0,0,2,0,2,2,0,0,0,0,14,0,14,14,2,0,2,2,95,23,72,72
Pop_Female,101,553,0,553,499,0,0,0,0,0,0,0,0,52,0,52,52,0,0,0,0,0,0,0,0,0,0,0,0,56,0,56,56
Pop_Total,102,2519,18,2501,2317,34,0,34,16,0,0,0,0,118,21,97,87,0,0,0,0,147,13,134,134,94,0,94,94,332,31,301,265
Pop_Male,102,1345,0,1345,1200,0,0,0,0,0,0,0,0,19,0,19,9,0,0,0,0,57,0,57,57,84,0,84,84,209,0,209,173
Pop_Female,102,1174,18,1156,1117,34,0,34,16,0,0,0,0,99,21,78,78,0,0,0,0,90,13,77,77,10,0,10,10,123,31,92,92
Pop_Total,200,48,0,48,44,9,0,9,0,0,0,0,0,30,0,30,30,0,0,0,0,0,0,0,0,0,0,0,0,8,0,8,8
Pop_Male,200,48,0,48,44,2,0,2,0,0,0,0,0,6,0,6,6,0,0,0,0,0,0,0,0,0,0,0,0,8,0,8,8
Pop_Female,200,0,0,0,0,7,0,7,0,0,0,0,0,24,0,24,24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
# Calculate total number of individauls from all groups by educational attainment and insert into table
ed_data_raw.insert(1, 'all', ed_data_raw["white"] + ed_data_raw["black"] + ed_data_raw["aian"] + ed_data_raw["asian"] +
                   ed_data_raw["nhpi"] + ed_data_raw["other"] + ed_data_raw["multi"] + ed_data_raw["hisp"])

ed_data_raw.insert(2, 'all_no', ed_data_raw["white_no"] + ed_data_raw["black_no"] + ed_data_raw["aian_no"] + ed_data_raw["asian_no"] +
                   ed_data_raw["nhpi_no"] + ed_data_raw["other_no"] + ed_data_raw["multi_no"] + ed_data_raw["hisp_no"])

ed_data_raw.insert(3, 'all_hs', ed_data_raw["white_hs"] + ed_data_raw["black_hs"] + ed_data_raw["aian_hs"] + ed_data_raw["asian_hs"] +
                   ed_data_raw["nhpi_hs"] + ed_data_raw["other_hs"] + ed_data_raw["multi_hs"] + ed_data_raw["hisp_hs"])

ed_data_raw.insert(4, 'all_col', ed_data_raw["white_col"] + ed_data_raw["black_col"] + ed_data_raw["aian_col"] + ed_data_raw["asian_col"] +
                   ed_data_raw["nhpi_col"] + ed_data_raw["other_col"] + ed_data_raw["multi_col"] + ed_data_raw["hisp_col"])

# Display the updated data table
ed_data_raw.head(20)

,tract,all,all_no,all_hs,all_col,white,white_no,white_hs,white_col,black,black_no,black_hs,black_col,aian,aian_no,aian_hs,aian_col,asian,asian_no,asian_hs,asian_col,nhpi,nhpi_no,nhpi_hs,nhpi_col,other,other_no,other_hs,other_col,multi,multi_no,multi_hs,multi_col,hisp,hisp_no,hisp_hs,hisp_col
label,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Pop_Total,101,1237,46,1191,1103,1016,23,993,905,0,0,0,0,0,0,0,0,54,0,54,54,0,0,0,0,14,0,14,14,2,0,2,2,151,23,128,128
Pop_Male,101,576,46,530,496,463,23,440,406,0,0,0,0,0,0,0,0,2,0,2,2,0,0,0,0,14,0,14,14,2,0,2,2,95,23,72,72
Pop_Female,101,661,0,661,607,553,0,553,499,0,0,0,0,0,0,0,0,52,0,52,52,0,0,0,0,0,0,0,0,0,0,0,0,56,0,56,56
Pop_Total,102,3244,83,3161,2913,2519,18,2501,2317,34,0,34,16,0,0,0,0,118,21,97,87,0,0,0,0,147,13,134,134,94,0,94,94,332,31,301,265
Pop_Male,102,1714,0,1714,1523,1345,0,1345,1200,0,0,0,0,0,0,0,0,19,0,19,9,0,0,0,0,57,0,57,57,84,0,84,84,209,0,209,173
Pop_Female,102,1530,83,1447,1390,1174,18,1156,1117,34,0,34,16,0,0,0,0,99,21,78,78,0,0,0,0,90,13,77,77,10,0,10,10,123,31,92,92
Pop_Total,200,95,0,95,82,48,0,48,44,9,0,9,0,0,0,0,0,30,0,30,30,0,0,0,0,0,0,0,0,0,0,0,0,8,0,8,8
Pop_Male,200,64,0,64,58,48,0,48,44,2,0,2,0,0,0,0,0,6,0,6,6,0,0,0,0,0,0,0,0,0,0,0,0,8,0,8,8
Pop_Female,200,31,0,31,24,0,0,0,0,7,0,7,0,0,0,0,0,24,0,24,24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
# Reset index
ed_data_raw = ed_data_raw.reset_index()

# Display the updated data table
ed_data_raw.head(20)

,label,tract,all,all_no,all_hs,all_col,white,white_no,white_hs,white_col,black,black_no,black_hs,black_col,aian,aian_no,aian_hs,aian_col,asian,asian_no,asian_hs,asian_col,nhpi,nhpi_no,nhpi_hs,nhpi_col,other,other_no,other_hs,other_col,multi,multi_no,multi_hs,multi_col,hisp,hisp_no,hisp_hs,hisp_col
0,Pop_Total,101,1237,46,1191,1103,1016,23,993,905,0,0,0,0,0,0,0,0,54,0,54,54,0,0,0,0,14,0,14,14,2,0,2,2,151,23,128,128
1,Pop_Male,101,576,46,530,496,463,23,440,406,0,0,0,0,0,0,0,0,2,0,2,2,0,0,0,0,14,0,14,14,2,0,2,2,95,23,72,72
2,Pop_Female,101,661,0,661,607,553,0,553,499,0,0,0,0,0,0,0,0,52,0,52,52,0,0,0,0,0,0,0,0,0,0,0,0,56,0,56,56
3,Pop_Total,102,3244,83,3161,2913,2519,18,2501,2317,34,0,34,16,0,0,0,0,118,21,97,87,0,0,0,0,147,13,134,134,94,0,94,94,332,31,301,265
4,Pop_Male,102,1714,0,1714,1523,1345,0,1345,1200,0,0,0,0,0,0,0,0,19,0,19,9,0,0,0,0,57,0,57,57,84,0,84,84,209,0,209,173
5,Pop_Female,102,1530,83,1447,1390,1174,18,1156,1117,34,0,34,16,0,0,0,0,99,21,78,78,0,0,0,0,90,13,77,77,10,0,10,10,123,31,92,92
6,Pop_Total,200,95,0,95,82,48,0,48,44,9,0,9,0,0,0,0,0,30,0,30,30,0,0,0,0,0,0,0,0,0,0,0,0,8,0,8,8
7,Pop_Male,200,64,0,64,58,48,0,48,44,2,0,2,0,0,0,0,0,6,0,6,6,0,0,0,0,0,0,0,0,0,0,0,0,8,0,8,8
8,Pop_Female,200,31,0,31,24,0,0,0,0,7,0,7,0,0,0,0,0,24,0,24,24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,Pop_Total,202,3129,15,3114,2772,2684,15,2669,2356,47,0,47,18,0,0,0,0,112,0,112,112,18,0,18,18,81,0,81,81,12,0,12,12,175,0,175,175


In [18]:
# Create copy of table
total_pop = ed_data_raw.copy()

# Delete any unneeded rows
total_pop = total_pop[~total_pop["label"].str.startswith('Pop_Male')]
total_pop = total_pop[~total_pop["label"].str.startswith('Pop_Female')]

# Reset index
total_pop = total_pop.reset_index(drop=True)

# Delete label column
total_pop.drop(['label'], axis=1, inplace=True)

# Export to csv
total_pop.to_csv('data/total_pop_data.csv', index=True)

# Display the total data table
total_pop.head(20)

,tract,all,all_no,all_hs,all_col,white,white_no,white_hs,white_col,black,black_no,black_hs,black_col,aian,aian_no,aian_hs,aian_col,asian,asian_no,asian_hs,asian_col,nhpi,nhpi_no,nhpi_hs,nhpi_col,other,other_no,other_hs,other_col,multi,multi_no,multi_hs,multi_col,hisp,hisp_no,hisp_hs,hisp_col
0,101,1237,46,1191,1103,1016,23,993,905,0,0,0,0,0,0,0,0,54,0,54,54,0,0,0,0,14,0,14,14,2,0,2,2,151,23,128,128
1,102,3244,83,3161,2913,2519,18,2501,2317,34,0,34,16,0,0,0,0,118,21,97,87,0,0,0,0,147,13,134,134,94,0,94,94,332,31,301,265
2,200,95,0,95,82,48,0,48,44,9,0,9,0,0,0,0,0,30,0,30,30,0,0,0,0,0,0,0,0,0,0,0,0,8,0,8,8
3,202,3129,15,3114,2772,2684,15,2669,2356,47,0,47,18,0,0,0,0,112,0,112,112,18,0,18,18,81,0,81,81,12,0,12,12,175,0,175,175
4,300,4802,4,4798,4425,3768,0,3768,3519,111,0,111,18,0,0,0,0,301,4,297,266,0,0,0,0,38,0,38,38,163,0,163,163,421,0,421,421
5,400,1328,172,1156,1042,807,79,728,665,32,4,28,14,0,0,0,0,285,8,277,260,0,0,0,0,5,1,4,4,42,0,42,36,157,80,77,63
6,501,3110,38,3072,2795,2155,0,2155,2012,233,36,197,146,0,0,0,0,359,0,359,305,0,0,0,0,19,1,18,18,168,0,168,168,176,1,175,146
7,501,3145,42,3103,2680,2252,0,2252,2049,151,12,139,82,44,0,44,44,94,0,94,94,0,0,0,0,89,15,74,46,136,0,136,119,379,15,364,246
8,600,3970,233,3737,3195,2827,37,2790,2624,411,132,279,39,0,0,0,0,223,0,223,223,0,0,0,0,66,20,46,46,111,12,99,62,332,32,300,201
9,702,2975,162,2813,2342,1979,85,1894,1713,322,26,296,92,15,0,15,0,134,0,134,119,0,0,0,0,21,0,21,21,237,0,237,217,267,51,216,180


In [19]:
# Create copy of table
male_pop = ed_data_raw.copy()

# Delete any unneeded rows
male_pop = male_pop[~male_pop["label"].str.startswith('Pop_Total')]
male_pop = male_pop[~male_pop["label"].str.startswith('Pop_Female')]

# Reset index
male_pop = male_pop.reset_index(drop=True)

# Delete label column
male_pop.drop(['label'], axis=1, inplace=True)

# Export to csv
male_pop.to_csv('data/male_pop_data.csv', index=False)

# Display the total data table
male_pop.head(20)

,tract,all,all_no,all_hs,all_col,white,white_no,white_hs,white_col,black,black_no,black_hs,black_col,aian,aian_no,aian_hs,aian_col,asian,asian_no,asian_hs,asian_col,nhpi,nhpi_no,nhpi_hs,nhpi_col,other,other_no,other_hs,other_col,multi,multi_no,multi_hs,multi_col,hisp,hisp_no,hisp_hs,hisp_col
0,101,576,46,530,496,463,23,440,406,0,0,0,0,0,0,0,0,2,0,2,2,0,0,0,0,14,0,14,14,2,0,2,2,95,23,72,72
1,102,1714,0,1714,1523,1345,0,1345,1200,0,0,0,0,0,0,0,0,19,0,19,9,0,0,0,0,57,0,57,57,84,0,84,84,209,0,209,173
2,200,64,0,64,58,48,0,48,44,2,0,2,0,0,0,0,0,6,0,6,6,0,0,0,0,0,0,0,0,0,0,0,0,8,0,8,8
3,202,1358,0,1358,1241,1091,0,1091,1003,29,0,29,0,0,0,0,0,54,0,54,54,0,0,0,0,81,0,81,81,0,0,0,0,103,0,103,103
4,300,2352,4,2348,2119,1829,0,1829,1663,63,0,63,0,0,0,0,0,158,4,154,154,0,0,0,0,19,0,19,19,74,0,74,74,209,0,209,209
5,400,483,4,479,432,308,0,308,283,25,4,21,7,0,0,0,0,72,0,72,64,0,0,0,0,2,0,2,2,28,0,28,28,48,0,48,48
6,501,1575,38,1537,1424,1059,0,1059,1018,95,36,59,16,0,0,0,0,241,0,241,241,0,0,0,0,17,1,16,16,15,0,15,15,148,1,147,118
7,501,1436,24,1412,1240,1015,0,1015,969,96,12,84,45,16,0,16,16,36,0,36,36,0,0,0,0,45,6,39,17,42,0,42,25,186,6,180,132
8,600,1582,159,1423,1204,1233,37,1196,1116,173,104,69,0,0,0,0,0,31,0,31,31,0,0,0,0,9,9,0,0,61,0,61,40,75,9,66,17
9,702,1183,124,1059,884,843,73,770,672,89,0,89,27,0,0,0,0,57,0,57,42,0,0,0,0,0,0,0,0,122,0,122,122,72,51,21,21


In [20]:
# Create copy of table
female_pop = ed_data_raw.copy()

# Delete any unneeded rows
female_pop = female_pop[~female_pop["label"].str.startswith('Pop_Total')]
female_pop = female_pop[~female_pop["label"].str.startswith('Pop_Male')]

# Reset index
female_pop = female_pop.reset_index(drop=True)

# Delete label column
female_pop.drop(['label'], axis=1, inplace=True)

# Export to csv
female_pop.to_csv('data/female_pop_data.csv', index=False)

# Display the total data table
female_pop.head(20)

,tract,all,all_no,all_hs,all_col,white,white_no,white_hs,white_col,black,black_no,black_hs,black_col,aian,aian_no,aian_hs,aian_col,asian,asian_no,asian_hs,asian_col,nhpi,nhpi_no,nhpi_hs,nhpi_col,other,other_no,other_hs,other_col,multi,multi_no,multi_hs,multi_col,hisp,hisp_no,hisp_hs,hisp_col
0,101,661,0,661,607,553,0,553,499,0,0,0,0,0,0,0,0,52,0,52,52,0,0,0,0,0,0,0,0,0,0,0,0,56,0,56,56
1,102,1530,83,1447,1390,1174,18,1156,1117,34,0,34,16,0,0,0,0,99,21,78,78,0,0,0,0,90,13,77,77,10,0,10,10,123,31,92,92
2,200,31,0,31,24,0,0,0,0,7,0,7,0,0,0,0,0,24,0,24,24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,202,1771,15,1756,1531,1593,15,1578,1353,18,0,18,18,0,0,0,0,58,0,58,58,18,0,18,18,0,0,0,0,12,0,12,12,72,0,72,72
4,300,2450,0,2450,2306,1939,0,1939,1856,48,0,48,18,0,0,0,0,143,0,143,112,0,0,0,0,19,0,19,19,89,0,89,89,212,0,212,212
5,400,845,168,677,610,499,79,420,382,7,0,7,7,0,0,0,0,213,8,205,196,0,0,0,0,3,1,2,2,14,0,14,8,109,80,29,15
6,501,1535,0,1535,1371,1096,0,1096,994,138,0,138,130,0,0,0,0,118,0,118,64,0,0,0,0,2,0,2,2,153,0,153,153,28,0,28,28
7,501,1709,18,1691,1440,1237,0,1237,1080,55,0,55,37,28,0,28,28,58,0,58,58,0,0,0,0,44,9,35,29,94,0,94,94,193,9,184,114
8,600,2388,74,2314,1991,1594,0,1594,1508,238,28,210,39,0,0,0,0,192,0,192,192,0,0,0,0,57,11,46,46,50,12,38,22,257,23,234,184
9,702,1792,38,1754,1458,1136,12,1124,1041,233,26,207,65,15,0,15,0,77,0,77,77,0,0,0,0,21,0,21,21,115,0,115,95,195,0,195,159


In [21]:
#text = "Census Tract 13.02, District of Columbia, District of Columbia"

#tract = ''.join(re.findall('[0-9]*\.?[0-9]*', text))

#print(tract)